**Description**: Analyzes BERT *and* GPT-2 accuracies for a given number of test
observations (200 or 500). Currently just 1 chain. Need to increase that, but it's too
slow :-(

**Estimated runtime**: ~1 hour (Multi-core isn't working on my machine. Set
`cores=4` in the `model.fit` call if it works for you.)

Note: I'm still learning how to do this type of analysis.

In [ ]:
import os
from typing import Literal

import polars as pl
import seaborn as sns

import utils

In [ ]:
sns.set_theme(style="darkgrid")

# Configuration

Which number of test observations do you want to analyze?

In [ ]:
num_test: Literal[200, 500] = 500

# Load data

In [ ]:
def load(accuracies_home_dir: str = "accuracies_from_paper", num_test: int = num_test):
    dfs: list[pl.DataFrame] = []
    for model in utils.lm_type_to_name.keys():
        accuracies_dir = os.path.join(accuracies_home_dir, str(num_test), model)
        df = utils.load_num_correct(accuracies_dir, num_test).with_columns(
            model=pl.lit(model)
        )
        df = df.select("model").with_columns(df.select(pl.exclude("model")))
        dfs.append(df)
    return pl.concat(dfs)

In [ ]:
num_correct_df = load()

In [ ]:
equation = "num_correct ~ method + (1|model) + (1|dataset/pair)"
id_vars = ["pair", "model", "dataset"]

In [ ]:
model_control, summary_control, az_summary_control = utils.stat_model(
    num_correct_df,
    treatment="extra",
    control="base",
    equation=equation,
    id_vars=id_vars,
)

In [ ]:
print(model_control)

In [ ]:
model_bias, summary_bias, az_summary_bias = utils.stat_model(
    num_correct_df,
    treatment="test",
    control="extra",
    equation=equation,
    id_vars=id_vars,
)

In [ ]:
print(model_bias)